In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
def analise_estoque(filial):
    df_saldo_estoque = pd.read_excel(f'planilhas/estoque/saldo_estoque_{filial}.xlsx', header=11, usecols=('B,C,F,G,H,I,J,O,P,Q,S,U,X'))[0:-3]
    df_saldo_estoque = df_saldo_estoque.set_axis(['ean','produto','laboratorio','grupo','curva','estoque_minimo','demanda','estoque','preco_custo','preco_venda','lucro','preco_custo_total','preco_venda_total'], axis=1)
    df_saldo_estoque['grupo'] = df_saldo_estoque['grupo'].astype(int)
    df_saldo_estoque['preco_custo_total'] = df_saldo_estoque['preco_custo_total'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['preco_custo'] = df_saldo_estoque['preco_custo'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['estoque_minimo'] = pd.to_numeric(df_saldo_estoque['estoque_minimo'], errors='coerce')
    df_saldo_estoque['preco_custo_total'] = pd.to_numeric(df_saldo_estoque['preco_custo_total'], errors='coerce')
    df_saldo_estoque['preco_custo'] = pd.to_numeric(df_saldo_estoque['preco_custo'], errors='coerce')
    df_saldo_estoque['ean'] = df_saldo_estoque['ean'].fillna(0).astype(int)

    
    df_saldo_estoque_filtrado = df_saldo_estoque.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_filtrado.query('estoque == 0')
    df_faltas['valor_faltas'] = np.where((df_faltas['estoque_minimo'] - df_faltas['estoque']) < 0,0,((df_faltas['estoque_minimo'] - df_faltas['estoque'])*df_faltas['preco_custo']))
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    
    cadastro_produtos = pd.read_excel("planilhas/cadastro_produtos.xlsx", header=14, usecols=('C,D,H,I,J,K,O,Q,S,W'))
    cadastro_produtos = cadastro_produtos.set_axis(['codigo','produto','fator','lista','icms','ean','tipo','ncm','departamento','categoria'], axis=1)
    cadastro_produtos['ncm'] = cadastro_produtos['ncm'].astype(str)
    df_merged = df_saldo_estoque.merge(cadastro_produtos, on="produto", how="right", suffixes=('_estoque', '_cadastro'))
    
    return valor_em_estoque, valor_faltas, df_merged

In [5]:
valor_em_estoque, valor_faltas, df_saldo_estoque = analise_estoque('004')

In [6]:
df_saldo_estoque

,ean,produto,laboratorio,grupo,curva,estoque_minimo,demanda,estoque,preco_custo,preco_venda,lucro,preco_custo_total,preco_venda_total
0,0,!AG PROMO COPO 400ML LARANJA,NaN,15,D / Q,0,0.0,0.0,0.99,"1,01","0,02",0.00,"0,00"
1,0,!AG PROMO MOCHILA POLIESTER L,NaN,15,D / Q,0,0.0,12.0,0.99,"1,01","0,02",11.88,"12,12"
2,0,!AG PROMO NECESSAIRE PLASTICA,NaN,15,D / Q,0,0.0,0.0,0.99,"1,01","0,02",0.00,"0,00"
3,-2147483648,!BANNER THERAPI,NaN,15,D / Q,0,0.0,0.0,0.99,"1,00","0,01",0.00,"0,00"
4,-2147483648,!BOBINA CUPOM FISCAL 8CMX40M,NaN,9000,D / Q,0,0.0,109.0,0.99,"1,10","0,11",107.91,"119,90"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27327,-2147483648,ZYXEM 5MG/ML GTS 20ML,UCB,3000,D / Q,0,0.0,0.0,43.77,"92,91","49,14",0.00,"0,00"
27328,0,ZZZZ PRODUTO INATIVADO,NaN,9000,D / Q,0,0.0,0.0,1.00,"6,88","5,88",0.00,"0,00"
27329,-2147483648,ZZZZCARBOLITIUM 300MG 60CP,EUROFARMA,3000,D / Q,0,0.0,0.0,40.21,"53,50","13,29",0.00,"0,00"
27330,0,ZZZZZQUETIAPINA 100MG 30CP REV,EMS,4000,D / Q,0,0.0,0.0,19.78,"219,00","199,22",0.00,"0,00"


In [30]:
cadastro_produtos = pd.read_excel("planilhas/cadastro_produtos.xlsx", header=14, usecols=('C,D,H,I,J,K,O,Q,S,W'))
cadastro_produtos = cadastro_produtos.set_axis(['codigo','produto','fator','lista','icms','ean','tipo','ncm','departamento','categoria'], axis=1)
cadastro_produtos['ncm'] = cadastro_produtos['ncm'].astype(str)
cadastro_produtos

,codigo,produto,fator,lista,icms,ean,tipo,ncm,departamento,categoria
0,141136,!AG PROMO COPO 400ML LARANJA,1.0,NEGATIVA,"12,00",NaN,NaN,39241000.0,te,NaN
1,141137,!AG PROMO MOCHILA POLIESTER L,1.0,NEGATIVA,"12,00",NaN,NaN,42029200.0,NaN,NaN
2,141134,!AG PROMO NECESSAIRE PLASTICA,1.0,NEGATIVA,"12,00",NaN,NaN,42029200.0,NaN,NaN
3,139207,!BANNER THERAPI,1.0,NEUTRA,"17,00",7.893590e+12,NaN,49111090.0,NaN,NaN
4,133397,!BOBINA CUPOM FISCAL 8CMX40M,30.0,NEUTRA,"17,00",7.898938e+12,NaN,48102290.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
27429,136146,ZZZZCARBOLITIUM 300MG 60CP REV,1.0,NEGATIVA,"17,00",1.000000e+12,NaN,30049099.0,NaN,NaN
27430,131310,ZZZZZQUETIAPINA 100MG 30CP REV,1.0,NEGATIVA,"17,00",NaN,NaN,30049079.0,MEDICAMENTOS,NaN
27431,85,ZZZZZZANADOR 500MG ENV 4CP,1.0,NEGATIVA,"17,00",7.896026e+13,NaN,30049069.0,NaN,NaN
27432,Total de Itens:,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN


In [39]:
df_merged = df_saldo_estoque.merge(cadastro_produtos, on="produto", how="right", suffixes=('_estoque', '_cadastro'))


In [42]:
df_merged.query("departamento == 'te'")

,ean_estoque,produto,laboratorio,grupo,curva,estoque_minimo,demanda,estoque,preco_custo,preco_venda,...,preco_venda_total,codigo,fator,lista,icms,ean_cadastro,tipo,ncm,departamento,categoria
0,NaN,!AG PROMO COPO 400ML LARANJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,141136,1.0,NEGATIVA,"12,00",NaN,NaN,39241000.0,te,NaN
